In [1]:
import os
import sys
from datetime import datetime

In [2]:
# -------------------------------------------------------------
# Environment setup
# -------------------------------------------------------------
BASE_DIR = os.path.abspath("../")
sys.path.append(BASE_DIR)

# -------------------------------------------------------------
# Configuration
# -------------------------------------------------------------
from src.utils.config import load_config
config = load_config(base_dir=BASE_DIR)

In [3]:

import pandas as pd
import numpy as np
import dask.dataframe as dd

import torch
import torch.nn.functional as F
from itertools import product
import time
from tqdm import tqdm


## Recovery sessions

Dado:
Sesion de TX
s = {cliente, timestamp, Monto}

    Cada cliente tiene una secuencia de sesiones de TX (S)
        S = { s1, s2 , s3, ..., sn}          n: # de sesiones de cada cliente

In [4]:
file_path = config["paths"]["session_records"]
data_sessions = pd.read_parquet(file_path)
# data_sessions = dd.read_parquet(file_path)
data_sessions.head(2)

,id,client_id,mcc,amount_sol,client_age,client_gender,agency_ubigeo,debit_type,timestamp,mccg,windows_time,day_of_week,hour,turn_of_day,amount_log
0,27634759,XxgXt1gGsxk=,5541,100.0,53,M,150140.0,TD,2017-02-05 15:45:25-05:00,8,2017-01-30,6,15,2,4.615121
1,27634839,by4Jhi0/BVI=,5541,20.0,49,M,70102.0,TD,2017-05-19 13:07:38-05:00,8,2017-05-15,4,13,2,3.044522


In [5]:
data_sessions.dtypes

id                                                int64
client_id                                        object
mcc                                               int64
amount_sol                                      float64
client_age                                        int64
client_gender                                    object
agency_ubigeo                                   float64
debit_type                                       object
timestamp        datetime64[ns, pytz.FixedOffset(-300)]
mccg                                              int64
windows_time                                     object
day_of_week                                       int64
hour                                              int64
turn_of_day                                       int64
amount_log                                      float64
dtype: object

In [6]:
dimension_names = ["day_of_week", "turn_of_day", "mccg"]

dimension_dict = {}

for dim in dimension_names:
    max_val = int(data_sessions[dim].max())
    dimension_dict[dim] = {
        "size": max_val + 1,
        "elements": list(range(max_val + 1))
    }

dimension_sizes = [dimension_dict[d]["size"] for d in dimension_names]
total_size = np.prod(dimension_sizes)

d1_size, d2_size, d3_size = dimension_sizes   # sizes for the 3D tensor
dimension_dict


{'day_of_week': {'size': 7, 'elements': [0, 1, 2, 3, 4, 5, 6]},
 'turn_of_day': {'size': 4, 'elements': [0, 1, 2, 3]},
 'mccg': {'size': 14,
  'elements': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]}}

In [7]:
def fun_user_footprint(user_data):
    """ User footprint building, only for 3D"""
    user_footprint = {}
    for idx, row in user_data.iterrows():

        # Columns for temporal windows
        windows_time = row["windows_time"]

        # Dimensions
        dim_1 = row["day_of_week"]
        dim_2 = row["turn_of_day"]
        dim_3 = row["mccg"]

        # Position
        position = (
            ((dim_3 - 1) * (d2_size * d1_size)) 
            + ((dim_2 - 1) * (d1_size))
            + (dim_1)
        )  # Only for 3D
        # dimension_values = [v+1 for v in row[dimension_names]]
        # position = np.prod(dimension_values)

        # Values
        # value = row["amount_sol"]
        value = row["amount_log"]

        if not (windows_time in user_footprint):
            user_footprint[windows_time] = np.array([0] * total_size)
            
        # user_footprint[windows_time][position] += value
        old_val = user_footprint[windows_time][position]
        new_val = np.log(1 + (np.exp(old_val) - 1) + (np.exp(value) - 1))
        user_footprint[windows_time][position] = new_val
    return user_footprint


In [8]:

# kernel = np.ones((3, 3, 3), dtype=np.float32) / 27
kernel = np.ones((3, 3, 3), dtype=np.float32)
kernel_tensor = torch.tensor(kernel).unsqueeze(0).unsqueeze(0).float()

def conv3d_output_shape(input_shape, kernel_shape, stride=(1,1,1), padding=(0,0,0)):
    return tuple(
        ((i + 2*p - k) // s) + 1
        for i, k, s, p in zip(input_shape, kernel_shape, stride, padding)
    )

def convolve_vector(vector):
    vector = vector.astype(float)
    tensor_3d = vector.reshape(dimension_sizes)
    input_tensor = torch.tensor(tensor_3d).unsqueeze(0).unsqueeze(0).float()
    output_tensor = F.conv3d(input_tensor, kernel_tensor, padding=0)
    output_np = output_tensor.squeeze().numpy() 
    result = output_np.flatten()
    return result



In [9]:
# For footprints
elements_lists = [dimension_dict[key]['elements'] for key in dimension_names[::-1]]
footprints_columns = [f"d{d}_{t}_{m}" for d, t, m in product(*elements_lists)]
footprints_ids = ['customer_id','windows_time','profile_id']
title = footprints_ids + footprints_columns
title = ','.join(title)+'\n'

# For convolution
output_shape = conv3d_output_shape(dimension_sizes, kernel.shape, stride = (1, 1, 1))
n_columns = np.prod(output_shape)
footprints_ids_conv = ['row_id']
title_individual_tensors = footprints_ids_conv + [f'resp_{i}' for i in range(n_columns)]
title_individual_tensors = ','.join(title_individual_tensors)+'\n'


In [10]:
n = 2
data_sessions_filtered = data_sessions.groupby('client_id').filter(lambda x: len(x) >= n)
grouped_data = dict(tuple(data_sessions_filtered.groupby("client_id")))

print("Total clients:", len(grouped_data))
client_id = '+++AhJk2QJM='
grouped_data[client_id]

Total clients: 1222055


,id,client_id,mcc,amount_sol,client_age,client_gender,agency_ubigeo,debit_type,timestamp,mccg,windows_time,day_of_week,hour,turn_of_day,amount_log
9350287,51513806,+++AhJk2QJM=,5812,26.00,32,M,40104.0,TD,2017-01-12 17:04:43-05:00,10,2017-01-09,3,17,2,3.295837
17105302,49543458,+++AhJk2QJM=,5812,21.00,32,M,40104.0,TD,2016-07-16 21:49:03-05:00,10,2016-07-11,5,21,3,3.091042
17337922,49807920,+++AhJk2QJM=,5812,128.00,32,M,40104.0,TD,2016-12-18 20:56:06-05:00,10,2016-12-12,6,20,3,4.859812
29550240,10337051,+++AhJk2QJM=,6011,50.00,32,M,40104.0,TD,2017-01-15 23:44:36-05:00,10,2017-01-09,6,23,3,3.931826
36536038,30300716,+++AhJk2QJM=,5411,6.62,32,M,40104.0,TD,2016-07-18 18:02:22-05:00,8,2016-07-18,0,18,3,2.030776
40248402,35637838,+++AhJk2QJM=,5813,150.00,32,M,40104.0,TD,2016-07-16 07:38:22-05:00,10,2016-07-11,5,7,1,5.017280
49077578,49543457,+++AhJk2QJM=,5812,167.00,32,M,40104.0,TD,2016-07-16 20:00:45-05:00,10,2016-07-11,5,20,3,5.123964
51391783,10337050,+++AhJk2QJM=,6011,100.00,32,M,40104.0,TD,2016-07-16 17:02:51-05:00,10,2016-07-11,5,17,2,4.615121
68093487,35637839,+++AhJk2QJM=,5813,145.00,32,M,40104.0,TD,2016-12-04 05:49:31-05:00,10,2016-11-28,6,5,0,4.983607
83368143,35636932,+++AhJk2QJM=,5813,150.00,32,M,40104.0,TD,2016-07-23 04:07:16-05:00,10,2016-07-18,5,4,0,5.017280


In [11]:
set(dimension_sizes)

{4, 7, 14}

In [12]:
# --- Timing: START ---
start_time = time.time()
start_time_str = time.ctime(start_time)

# --- File Setup (Existing Code) ---
individual_footprint = config["paths"]["footprints"]
os.makedirs(os.path.dirname(individual_footprint), exist_ok=True)
fw=open(individual_footprint,'w')
fw.write(title)

individual_tensors = config["paths"]["tensors_convolution"]
os.makedirs(os.path.dirname(individual_tensors), exist_ok=True)
fw_tensors=open(individual_tensors,'w')
fw_tensors.write(title_individual_tensors)

# --- Processing Loop (Existing Code) ---
footprints = 0
for user_id, user_i in tqdm(grouped_data.items()):
    # user_footprint = fun_user_footprint_fast(user_i, d1_size, d2_size, d3_size, total_size)
    user_footprint = fun_user_footprint(user_i)
    user_footprint = dict(sorted(user_footprint.items()))
    profile_id = 0
    for windows_time, vector in user_footprint.items():
        # print(f"{windows_time}: {value}")
        # print(f"{windows_time}")

        # footprints
        value = [user_id, windows_time, str(profile_id)] + [str(v) for v in vector]
        value = ','.join(value)+'\n'
        fw.write(value)

        # footprints convoluted
        tensor_3d = convolve_vector(vector)
        # tensor_3d = [user_id, windows_time, str(profile_id)] + [str(v) for v in tensor_3d]
        row_id = user_id + "_" + str(profile_id)
        tensor_3d = [row_id] + [str(v) for v in tensor_3d]
        tensor_3d = ','.join(tensor_3d)+'\n'
        fw_tensors.write(tensor_3d)

        profile_id +=1
    footprints += profile_id
    fw.flush()
    fw_tensors.flush()
    # break;

# --- File Cleanup (Existing Code) ---
fw.close()
fw_tensors.close()

# --- Timing: END & Metrics Calculation ---
end_time = time.time()
end_time_str = time.ctime(end_time)
total_execution_seconds = end_time - start_time
total_execution_minutes = total_execution_seconds / 60

# --- Final Output ---
print(f"Quantity of footprints: {footprints}")
print("-" * 30)
print(f"⏰ Start time: {start_time_str}")
print(f"🏁 End time: {end_time_str}")
print(f"⏱️ Total execution time: {total_execution_seconds:.2f} seconds")
print(f"⏱️ Total execution time: {total_execution_minutes:.2f} minutes")




100%|██████████| 1222055/1222055 [1:01:40<00:00, 330.22it/s]


Quantity of footprints: 26283914
------------------------------
⏰ Start time: Thu Nov 27 14:05:04 2025
🏁 End time: Thu Nov 27 15:06:45 2025
⏱️ Total execution time: 3700.82 seconds
⏱️ Total execution time: 61.68 minutes


# end

In [ ]:
jkn

In [ ]:
import pandas as pd

In [ ]:
file_path = config["paths"]["session_records"]
data_sessions = pd.read_parquet(file_path)
data_sessions.head(2)

In [ ]:
grouped_data = dict(tuple(data_sessions.groupby("client_id")))

print("Total clients:", len(grouped_data))
client_id = '+++AhJk2QJM='
grouped_data[client_id]


In [ ]:
n = 2
data_sessions_filtered = data_sessions.groupby('client_id').filter(lambda x: len(x) >= n)
grouped_data = dict(tuple(data_sessions_filtered.groupby("client_id")))

print("Total clients:", len(grouped_data))
client_id = '+++AhJk2QJM='
grouped_data[client_id]

In [ ]:
file_path = config["paths"]["session_records"]
data_sessions = pd.read_parquet(file_path)
data_sessions.head(2)

n = 2
data_sessions_filtered = data_sessions.groupby('client_id').filter(lambda x: len(x) >= n)
grouped_data = dict(tuple(data_sessions_filtered.groupby("client_id")))

print("Total clients:", len(grouped_data))
client_id = '+++AhJk2QJM='
grouped_data[client_id]